The Fibonacci sequence is defined by the recurrence $F_n = F_{n - 1} + F_{n - 2}$. We explore how this sequence behaves with different $F_0$, and $F_1$. Beyond the trivial $F_0, F_1 = 0$, we note that regardless of our choice for these two seeds, seemingly, $F_n/F_{n - 1} \rightarrow 1.618 \ldots$ as $n \rightarrow \infty$.

Here, we define a function which will confirm that this tends to happen for random reals, say in the range $[-8, 8]$.

In [1]:
from functools import lru_cache  # to speed up computation

@lru_cache(maxsize=None)
def fib(n, f0, f1):
    prev, curr = f0, f1
    for _ in range(n):
        prev, curr = curr, prev + curr
    return prev

# approximate the limit at step n
def fib_lim(n, f0, f1):
    return fib(n, f0, f1)/fib(n - 1, f0, f1)

In [2]:
from random import uniform
from tabulate import tabulate

data = []
for _ in range(10):  # sample random choices for f0, f1
    f0, f1 = uniform(-8, 8), uniform(-8, 8)
    data.append([f0, f1, fib_lim(100, f0, f1)])
                                     
print(tabulate(data, headers=['f0', 'f1', 'limit']))

       f0        f1    limit
---------  --------  -------
-1.82604   -4.14088  1.61803
-6.1645     2.59453  1.61803
-3.96252    1.12992  1.61803
-2.18146    4.80843  1.61803
-1.96105    5.83017  1.61803
-7.30095   -2.39872  1.61803
-0.424836   4.90962  1.61803
-1.85648    4.9479   1.61803
-1.57878    5.49927  1.61803
-0.406386   5.49799  1.61803


The following outlines why this is mostly true. Let $L_n$ denote this partial limit, $F_{n + 1}/F_n$. See that

$$
L_n = \frac{F_{n} + F_{n - 1}}{F_n} = 1 + 1/L_{n - 1}
$$

If $L_n$ approaches some limit, then so does $L_{n - 1}$; call this limit $L$. Then $L^2 = L + 1$, which has the roots $\frac{1 \pm \sqrt{5}}{2}$. The limit must tend to a positve number since as we assume the limit approaches some limit (not zero as we've just shown), then all the terms must be the same sign. Hence, $L = \frac{1 + \sqrt{5}}{2} = \phi$.

There are some choices of the seeds that do not converge to this limit however:

In [3]:
from math import sqrt

phi = (1 + sqrt(5))/2

# W limit ourselves to 30 steps, here, for the sake of precision errors
# We'll prove it more rigorously, shorlty.
data = []
for f0 in range(-5, 5):
    if f0 == 0:
        continue  # skip f0 == 0, for divide by 0 error
    data.append([f0, -f0/phi, fib_lim(30, f0, -f0/phi)])
    
print(tabulate(data, headers=['f0', 'f1', 'limit']))

  f0         f1      limit
----  ---------  ---------
  -5   3.09017   -0.61808
  -4   2.47214   -0.618109
  -3   1.8541    -0.61806
  -2   1.23607   -0.618109
  -1   0.618034  -0.618109
   1  -0.618034  -0.618109
   2  -1.23607   -0.618109
   3  -1.8541    -0.61806
   4  -2.47214   -0.618109


To explore this suprise, first see that Fibonacci numbers form a vector space. That is, if $F_n$ and $G_n$ are Fibonacci sequences with different seeds, then $\alpha \cdot F_n + \beta \cdot G_n$ is also a Fibonacci sequence. This is because we can add two sequences, and scale sequences by a constant, and still get out a Fibonacci sequence.

Also, see that a Fibonacci sequence is uniquely defined by it's seeds.

Now, for a moment, let $F_n$ define the traditional $F_0, F_1 = 0, 1$ sequence. Also define $F^\prime_n = F_{n - 1}$, with $F_{-1} = 0$. And let $x_n$ define a Fibonacci sequence of arbitrary seeds. Then,

$$
x_0 = x_0 \cdot F^\prime_{0} + x_1 \cdot F_{0}
$$

and 

$$
x_1 = x_1 \cdot F^\prime_{1} + x_1 \cdot F_{1}
$$

It follows from the recurrence definition that

$$x_n = x_0 \cdot F^\prime_n + x_1 \cdot F_n$$

Let's now replace $F^\prime_n$ with $F_{n - 1}$, and look at the ratio $x_{n + 1}/x_n$:

$$\frac{x_0 \cdot F_n + x_1 \cdot F_{n + 1}}
{x_0 \cdot F_{n - 1} + x_1 \cdot F_n}
$$

Dividing out $F_n$ yields

$$\frac{x_0 + x_1 \cdot L_n}
{x_0 \cdot L_{n - 1}^{-1} + x_1}
$$

And we already have that $L_n \rightarrow \phi$ and $L_{n - 1} \rightarrow \phi$, so that 

$$x_n \rightarrow \frac{x_0 + x_1 \phi}{x_0 \phi^{-1} + x_1}
$$

which is $\phi$ since its denominator, multiplied by $\phi$, is the numerator.

The only problem is when the denominator is 0, or $x_1 = -x_0/\phi$ (see the most recent computation). In this case, it follows that $x_n = x_0 \cdot \phi^{-n}$, so that the limit is $1/\phi$.

In summary, Fibonacci sequences tend toward $\phi$, unless $F_1 = -F_0/\phi$, in which case the limit is $1/\phi$.